In [1]:
using Pkg

In [ ]:
Pkg.activate("..\\PEPS_TensorKit")

  Activating project at `C:\Users\ZXM\Julia\包环境\PEPS_TensorKit`


In [3]:
using MKL
using LinearAlgebra
LinearAlgebra.BLAS.set_num_threads(8)
using TensorOperations, TensorKit
using Statistics
import TensorKit.×
using JLD2
using Strided, FLoops
Strided.enable_threads()
@show Threads.nthreadpools()
@show Threads.nthreads()

Threads.nthreadpools() = 2
Threads.nthreads() = 1


1

In [4]:
include("../PEPS_TensorKit-main/iPEPS_Fermionic/iPEPS.jl")

check_qn (generic function with 2 methods)

In [5]:
include("../PEPS_TensorKit-main/CTMRG_Fermionic/CTMRG.jl")

update_env_bottom_2by2! (generic function with 1 method)

In [6]:
include("../PEPS_TensorKit-main/models/Hubbard_Z2U1.jl")

get_op_Hubbard (generic function with 1 method)

In [7]:
include("../PEPS_TensorKit-main/simple_update_Fermionic/simple_update.jl")

diag_proj_ru2ld! (generic function with 1 method)

In [27]:
include("../PEPS_TensorKit-main/Cal_Obs_Fermionic/Cal_Obs.jl")

_2siteObs_diagSite (generic function with 1 method)

In [9]:
para = Dict{Symbol,Any}()
para[:t] = 1.0
para[:U] = 8.0
para[:μ] = 4.0
para[:τlisSU] = [1.0, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0001]
# para[:τlis] = [1.0]
para[:minStep1τ] = 10
para[:maxStep1τ] = 100  # 对每个虚时步长 τ , 最多投影这么多步
para[:Dk] = 6  # Dkept in the simple udate
para[:χ] = 100  # env bond dimension
para[:CTMit] = 20  # CTMRG iteration times
para[:Etol] = 1e-6  # simple update 能量差小于 para[:Etol]*τ² 这个数就可以继续增大步长
para[:verbose] = 1
para[:TrotterOrder] = 1 # 用几阶Trotter分解,设为1或2
para[:CTMparallel] = true  # contract CTMRG env in parallel or not. Better with MKL??
para[:pspace] = Rep[ℤ₂×U₁]((0, 0) => 2, (1, 1 // 2) => 1, (1, -1 // 2) => 1);

In [10]:
pspace = Rep[ℤ₂×U₁]((0, 0) => 2, (1, 1 // 2) => 1, (1, -1 // 2) => 1)
aspacelr = Rep[ℤ₂×U₁]((0, 0) => 2)
aspacetb = Rep[ℤ₂×U₁]((0, 0) => 2)
Lx = 2
Ly = 2;

In [11]:
ipepsγλ = iPEPSΓΛ(pspace, aspacelr, aspacetb, Lx, Ly; dtype=Float64)

iPEPSΓΛ(_iPEPSΓΛ[_iPEPSΓΛ(TensorMap((Rep[ℤ₂ × U₁]((0, 0)=>2) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>2) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1)) ← (Rep[ℤ₂ × U₁]((0, 0)=>2) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>2))):
* Data for sector (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)) ← (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)):
[:, :, 1, 1, 1] =
 -0.2233936288784505    0.07923272310799745
  0.04862974692169264  -0.16841542106654336

[:, :, 2, 1, 1] =
 0.3200879045623859     0.16111698817607834
 0.026060879751602418  -0.0045129207698319795

[:, :, 1, 2, 1] =
 -0.1646855597199723   0.18243919962933944
  0.0898341792566131  -0.0015436366915120934

[:, :, 2, 2, 1] =
  0.0071460984009257406  -0.05590434972393989
 -0.1251288652006302      0.3017111923861023

[:, :, 1, 1, 2] =
 -0.23305748243399502  0.33805835614656304
 -0.12003196775572049  0.17458752201200986

[:, :, 2, 1, 2] =
 0.13702232241356174   0.08258754079800626
 0.2671031435329295   -0.2035656538269873

[:, :, 1, 2, 2] =
  0.152384

In [12]:
simple_update!(ipepsγλ, Hubbard_hij, para)

SU one step: 39.449483 seconds (39.28 M allocations: 1.882 GiB, 1.51% gc time, 99.88% compilation time)
Truncation error = 3.23168651074162, total imaginary time = 1.0
Estimated energy per site is -3.5176362160329457
=========== Step τ=1.0, iteration 1, total iteration 1 =======

SU one step: 0.028218 seconds (34.27 k allocations: 7.507 MiB, 20.65% compilation time)
Truncation error = 5.898119676965913, total imaginary time = 2.0
Estimated energy per site is -4.985679498266448
=========== Step τ=1.0, iteration 2, total iteration 2 =======

SU one step: 0.021643 seconds (27.59 k allocations: 5.919 MiB, 0.06% compilation time)
Truncation error = 5.451870749626859, total imaginary time = 3.0
Estimated energy per site is -4.906323336244505
=========== Step τ=1.0, iteration 3, total iteration 3 =======

SU one step: 0.019601 seconds (27.59 k allocations: 5.919 MiB, 0.05% compilation time)
Truncation error = 5.632630649041587, total imaginary time = 4.0
Estimated energy per site is -4.923910

In [13]:
ipeps = iPEPS(ipepsγλ)

iPEPS(AbstractTensorMap[TensorMap((Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (1, -1/2)=>2, (0, -1)=>1) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (1, -1/2)=>2, (0, -1)=>1) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1)) ← (Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1, (0, -1)=>2) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (0, 1)=>1, (1, -1/2)=>2))):
* Data for sector (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)) ← (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)):
[:, :, 1, 1, 1] =
 -0.3454461873902512

[:, :, 2, 1, 1] =
 -0.046368408893514505

[:, :, 1, 2, 1] =
 0.04636840889351005

[:, :, 2, 2, 1] =
 -0.34544618739022387
* Data for sector (Irrep[ℤ₂ × U₁](1, -1/2), Irrep[ℤ₂ × U₁](1, 1/2), Irrep[ℤ₂ × U₁](0, 0)) ← (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)):
[:, :, 1, 1, 1] =
 -0.04036465500808796  0.042470660672959765
 -0.00555472332855013  0.005530816669015738

[:, :, 2, 1, 1] =
 -0.0423339876705825     -0.04019473728791887
 -0.0057007343891655815  -0.005418050326160044

In [14]:
space(ipeps[1, 1])

(Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (1, -1/2)=>2, (0, -1)=>1) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (1, -1/2)=>2, (0, -1)=>1) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1)) ← (Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1, (0, -1)=>2) ⊗ Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (0, 1)=>1, (1, -1/2)=>2))

In [15]:
ipepsbar = bar(ipeps)

iPEPS(AbstractTensorMap[TensorMap((Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (1, -1/2)=>2, (0, -1)=>1)' ⊗ Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (1, -1/2)=>2, (0, -1)=>1)' ⊗ Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1)') ← (Rep[ℤ₂ × U₁]((0, 0)=>2, (1, 1/2)=>1, (1, -1/2)=>1, (0, -1)=>2)' ⊗ Rep[ℤ₂ × U₁]((0, 0)=>1, (1, 1/2)=>2, (0, 1)=>1, (1, -1/2)=>2)')):
* Data for sector (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)) ← (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)):
[:, :, 1, 1, 1] =
 -0.3454461873902512

[:, :, 2, 1, 1] =
 -0.046368408893514505

[:, :, 1, 2, 1] =
 0.04636840889351005

[:, :, 2, 2, 1] =
 -0.34544618739022387
* Data for sector (Irrep[ℤ₂ × U₁](1, 1/2), Irrep[ℤ₂ × U₁](1, -1/2), Irrep[ℤ₂ × U₁](0, 0)) ← (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)):
[:, :, 1, 1, 1] =
 0.04036465500808796  -0.042470660672959765
 0.00555472332855013  -0.005530816669015738

[:, :, 2, 1, 1] =
 0.0423339876705825     0.04019473728791887
 0.0057007343891655815  0.00541805032616004

In [16]:
envs = iPEPSenv(ipeps)

iPEPSenv(_iPEPSenv[_iPEPSenv(_corner(TensorMap((Rep[ℤ₂ × U₁]((0, 0)=>10, (1, 1/2)=>6, (0, 1)=>5, (1, -1/2)=>6, (0, -1)=>5, (1, 3/2)=>2, (1, -3/2)=>2)' ⊗ Rep[ℤ₂ × U₁]((0, 0)=>10, (1, 1/2)=>6, (0, 1)=>5, (1, -1/2)=>6, (0, -1)=>5, (1, 3/2)=>2, (1, -3/2)=>2)') ← ProductSpace{GradedSpace{ProductSector{Tuple{Z2Irrep, U1Irrep}}, TensorKit.SortedVectorDict{ProductSector{Tuple{Z2Irrep, U1Irrep}}, Int64}}, 0}()):
* Data for sector (Irrep[ℤ₂ × U₁](0, 0), Irrep[ℤ₂ × U₁](0, 0)) ← ():
 1.0                      0.14392647504784023     …   0.04907538341293396
 0.14250235639941622      4.0099039573813627e-7       0.013213339989967982
 1.1949360830831838e-14  -1.4021384216941344e-15     -0.004286392394864443
 1.1949360830831838e-14  -1.4021384216941344e-15     -0.004286392394864443
 0.14250235639939884      0.03320133414471987         0.006400500886140828
 0.12203721518797259      0.0012993008457530188   …   0.01725284648723124
 0.0485862789243967       0.00735978394792431         0.0076566842377779895


In [17]:
check_qn(ipeps, envs)

In [18]:
CTMRG!(ipeps, ipepsbar, envs, para[:χ], para[:CTMit]; parallel=para[:CTMparallel])

============ CTMRG iteration 1 / 20 =======================
Update left edge (contract column-1) truncation error 8.140243887690697e-13
Update right edge (contract column-2) truncation error 8.140243896376536e-13
Update left edge (contract column-2) truncation error 1.026866337040456e-12
Update right edge (contract column-1) truncation error 1.0268663373118143e-12
Left-right env: 83.261657 seconds (52.04 M allocations: 7.900 GiB, 4.22% gc time, 87.67% compilation time)
Update top edge (contract row-1) truncation error 1.0242456664162405e-12
Update bottom edge (contract row-2) truncation error 1.0242456655462115e-12
Update top edge (contract row-2) truncation error 1.2584547138683462e-12
Update bottom edge (contract row-1) truncation error 1.258454713886086e-12
Top-Bottom env: 15.484959 seconds (3.69 M allocations: 9.088 GiB, 17.34% gc time, 17.78% compilation time)
Convergence Check: 0.630251 seconds (290.94 k allocations: 17.240 MiB, 99.31% compilation time)
Continue CTMRG with norm d

In [19]:
check_qn(ipeps, envs)

In [20]:
GC.gc()

## Calculate Obs

In [21]:
site1Obs = ["N", "Sz"];                     # 计算这些单点观测量

In [22]:
site2Obs = ["hij", "SpSm", "NN", "SzSz"];   # 计算这些两点观测量

In [23]:
@floop for ind in CartesianIndices((Lx, Ly)) 
    (xx, yy) = Tuple(ind)
    Obs1si = Cal_Obs_1site(ipeps, ipepsbar, envs, site1Obs, para; site=[xx, yy], get_op=get_op_Hubbard)
    @show Obs1si
    @reduce doping += get(Obs1si, "N", NaN) 
    GC.gc()
    Obs2si_h = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx + 1, yy], get_op=get_op_Hubbard) 
    @show Obs2si_h 
    Obs2si_v = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx, yy + 1], get_op=get_op_Hubbard)
    @show Obs2si_v 
    @reduce Eg += (get(Obs2si_h, "hij", NaN) + get(Obs2si_v, "hij", NaN))  
    GC.gc()  
    # Obs2sidiag = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx+1, yy+1], get_op=get_op_Hubbard)  
    # @show Obs2sidiag  
    # Obs2sidiag = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx - 1, yy + 1], get_op=get_op_Hubbard)
end
GC.gc()

Fermionic! Calculating ["N", "Sz"] at site [1, 1]
Obs1si = Dict("N" => 0.9999999999999942, "Sz" => -0.3583736043654929)
Fermionic! Calculating ["hij", "SpSm", "NN", "SzSz"] at site [1, 1] and site [2, 1]
Obs2si_h = Dict("SzSz" => -0.14910988732056035, "hij" => -2.2544238428268337, "SpSm" => -0.11741962654146898, "NN" => 0.977377926230124)
Fermionic! Calculating ["hij", "SpSm", "NN", "SzSz"] at site [1, 1] and site [1, 2]
Obs2si_v = Dict("SzSz" => -0.14988746522430038, "hij" => -2.262681386654832, "SpSm" => -0.11974483985270337, "NN" => 0.9766789077947184)
Fermionic! Calculating ["N", "Sz"] at site [2, 1]
Obs1si = Dict("N" => 1.0000000000000042, "Sz" => 0.35837360436555105)
Fermionic! Calculating ["hij", "SpSm", "NN", "SzSz"] at site [2, 1] and site [3, 1]
Obs2si_h = Dict("SzSz" => -0.1494842877045796, "hij" => -2.2579383231289345, "SpSm" => -0.11860581565919961, "NN" => 0.9770722219092435)
Fermionic! Calculating ["hij", "SpSm", "NN", "SzSz"] at site [2, 1] and site [2, 2]
Obs2si_v = Di

In [24]:
doping = doping / (Lx * Ly)

1.0000000000000002

In [25]:
Eg = Eg / (Lx * Ly) + doping * para[:μ]

-0.5207125841739932

## Calculate correlation function

In [ ]:
let Rx=5,Ry=0,Gates=["NN"]

    pre_compute_env = pre_compute(ipeps, ipepsbar, envs, Rx, Ry)

    Obslong_v=
    Cal_Obs_v(ipeps, ipepsbar, envs, pre_compute_env, Gates, para,get_op=get_op_Hubbard, Rx; site=[1, 1])
    @show Obslong_v
end